In [1]:
import nlopt
import numpy as np

# Test data
x0 = np.array([100.0, 200.0, 300.0])
lb = np.array([50.0, 100.0, 150.0])
ub = np.array([200.0, 400.0, 600.0])
eval_dict = {"total_budget_unit": 600.0}


def test_eval_f(x, grad):
    if grad.size > 0:
        grad[:] = -np.ones_like(x)  # Gradient for maximization
    return {"objective": -np.sum(x), "gradient": -np.ones_like(x)}["objective"]


def test_eval_g_eq(x, grad):
    if grad.size > 0:
        grad[:] = np.ones_like(x)
    return {
        "constraints": np.sum(x) - eval_dict["total_budget_unit"],
        "jacobian": np.ones_like(x),
    }["constraints"]


# Create optimizer
opt = nlopt.opt(nlopt.LD_AUGLAG, len(x0))

# Set local optimizer
local_optimizer = nlopt.opt(nlopt.LD_SLSQP, len(x0))
local_optimizer.set_xtol_rel(1.0e-10)
opt.set_local_optimizer(local_optimizer)

# Set objective and constraints
opt.set_min_objective(test_eval_f)
opt.add_equality_constraint(test_eval_g_eq)

# Set bounds and options
opt.set_lower_bounds(lb)
opt.set_upper_bounds(ub)
opt.set_maxeval(1000)
opt.set_xtol_rel(1.0e-10)

# Run optimization
try:
    x = opt.optimize(x0)
    result = {
        "solution": x,
        "objective": opt.last_optimum_value(),
        "status": opt.last_optimize_result(),
    }
    print("\nOptimization Results:")
    print(f"Solution: {result['solution']}")
    print(f"Objective: {result['objective']}")
    print(f"Status: {result['status']}")
    print(f"Total spend: {np.sum(result['solution'])}")
except Exception as e:
    print(f"Optimization failed: {str(e)}")


Optimization Results:
Solution: [100. 200. 300.]
Objective: -600.0
Status: 3
Total spend: 600.0
